In [1]:
!pip install timm
!pip install --upgrade wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.6 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.13.10
    Uninstalling wandb-0.13.10:
      Successfully uninstalled wandb-0.13.10


In [2]:
import os
import gc
import cv2
import math
import copy
import time
import random

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Audio 
import torchaudio
from torchaudio.transforms import MelSpectrogram, Resample

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score

# For Image Models
import timm

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. 
Get your W&B access token from here: https://wandb.ai/authorize


In [4]:
CONFIG = {"seed": 2023,
          "epochs": 10,
          "model_name": "tf_efficientnet_b0_ns",
          "embedding_size": 768,
          "num_classes": 264,
          "train_batch_size": 64,
          "valid_batch_size": 128,
          "learning_rate": 1e-4,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 5,
          "n_accumulate": 1,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          "_wandb_kernel": "deb",
          # Audio Specific
          "sample_rate": 32_000,
          "max_time": 5,
          "n_mels": 224,
          "n_fft": 1024,
          }

In [5]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [6]:
ROOT_DIR = '../input/birdclef-2023'
TRAIN_DIR = '../input/birdclef-2023/train_audio'
TEST_DIR = '../input/birdclef-2023/test_soundscapes'

In [7]:
def get_train_file_path(filename):
    return f"{TRAIN_DIR}/{filename}"

In [8]:
df = pd.read_csv(f"{ROOT_DIR}/train_metadata.csv")
df['file_path'] = df['filename'].apply(get_train_file_path)
df.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename,file_path
0,abethr1,[],['song'],4.3906,38.2788,Turdus tephronotus,African Bare-eyed Thrush,Rolf A. de By,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/128013,abethr1/XC128013.ogg,../input/birdclef-2023/train_audio/abethr1/XC1...
1,abethr1,[],['call'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/363501,abethr1/XC363501.ogg,../input/birdclef-2023/train_audio/abethr1/XC3...
2,abethr1,[],['song'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/363502,abethr1/XC363502.ogg,../input/birdclef-2023/train_audio/abethr1/XC3...
3,abethr1,[],['song'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/363503,abethr1/XC363503.ogg,../input/birdclef-2023/train_audio/abethr1/XC3...
4,abethr1,[],"['call', 'song']",-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://www.xeno-canto.org/363504,abethr1/XC363504.ogg,../input/birdclef-2023/train_audio/abethr1/XC3...


In [9]:
wandb.login(key='e48a210dfb62389215fad02ce3ca3f0d8825de66')
run = wandb.init(project='BirdCLEF2023',
                 job_type='Visualization',
                 name='Audio Visualization',
                 anonymous='must')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: byeol5832. Use `wandb login --relogin` to force relogin


In [10]:
preview_table = wandb.Table(columns=['Audio', 'Label', 'Rating'])

temp_df = df.sample(5000).reset_index(drop=True)

for i in tqdm(range(len(temp_df))):
    row = temp_df.loc[i]
    audio = wandb.Audio(row.file_path, sample_rate=CONFIG['sample_rate'])
    preview_table.add_data(audio,
                           row.primary_label,
                           row.rating)

wandb.log({'Visualization': preview_table})
run.finish()

100%|██████████| 5000/5000 [00:45<00:00, 110.40it/s]


<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;"><a href="https://wandb.ai/dchanda/BirdCLEF23">View the Complete Table Here ⮕</a></span>

In [11]:
# This is just to display the W&B run page in this interactive session.
from IPython import display

# we create an IFrame and set the width and height
iF = display.IFrame(run.url, width=1080, height=720)
iF

In [12]:
encoder = LabelEncoder()
all_labels = sorted(df['primary_label'].values)
encoder.fit(all_labels)

df['primary_label'] = encoder.transform(df['primary_label'])

with open("le.pkl", "wb") as fp:
    joblib.dump(encoder, fp)

In [13]:
skf = StratifiedKFold(n_splits=CONFIG['n_fold'])

for fold, ( _, val_) in enumerate(skf.split(X=df, y=df.primary_label)):
    # meta data의 primary_label을 n_fold개로 나눈 것을 enumerate하면서 각 fold를 indexing 
      df.loc[val_ , "kfold"] = fold

In [14]:
class BirdCLEFDataset(Dataset):
    def __init__(self, df, target_sample_rate, max_time, image_transforms=None):
        self.file_paths = df['file_path'].values
        self.labels = df['primary_label'].values
        self.target_sample_rate = target_sample_rate
        num_samples = target_sample_rate * max_time
        self.num_samples = num_samples
        self.image_transforms = image_transforms
        
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, index):
        filepath = self.file_paths[index]
        audio, sample_rate = torchaudio.load(filepath)
        audio = self.to_mono(audio) # audio를 한 채널로 바꾸기 위해 평균냄
        
        if sample_rate != self.target_sample_rate:
            resample = Resample(sample_rate, self.target_sample_rate) # resampling이 뭘 하는건지 정확히 모르겠다 
            audio = resample(audio)
        
        if audio.shape[0] > self.num_samples:
            audio = self.crop_audio(audio) # num_samples까지만 crop 
            
        if audio.shape[0] < self.num_samples:
            audio = self.pad_audio(audio) # 앞뒤로 num_samples - shape 만큼 padding 하는듯 
            
        mel_spectogram = MelSpectrogram(sample_rate=self.target_sample_rate, 
                                        n_mels=CONFIG['n_mels'], 
                                        n_fft=CONFIG['n_fft'])
        mel = mel_spectogram(audio)
        
        label_onehot = torch.zeros(CONFIG['num_classes'])
        label_onehot[self.labels[index]] = 1
        label = torch.tensor(self.labels[index])
        
        # Convert to Image
        image = torch.stack([mel, mel, mel]) 
        
        # Normalize Image
        max_val = torch.abs(image).max()
        image = image / max_val
        
        return {
            "image": image, 
            "label": label,
            "label_onehot": label_onehot
        }
            
    def pad_audio(self, audio):
        pad_length = self.num_samples - audio.shape[0]
        last_dim_padding = (0, pad_length)
        audio = F.pad(audio, last_dim_padding)
        return audio
        
    def crop_audio(self, audio):
        return audio[:self.num_samples]
        
    def to_mono(self, audio):
        return torch.mean(audio, axis=0)

In [15]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'

In [16]:
class BirdCLEFModel(nn.Module):
    def __init__(self, model_name, embedding_size, pretrained=True):
        super(BirdCLEFModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        self.pooling = GeM()
        self.embedding = nn.Linear(in_features, embedding_size)
        self.fc = nn.Linear(embedding_size, CONFIG['num_classes'])

    def forward(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        embedding = self.embedding(pooled_features)
        output = self.fc(embedding)
        return output
    
model = BirdCLEFModel(CONFIG['model_name'], CONFIG['embedding_size'])
model.to(CONFIG['device']);

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b0_ns-c0e6a31c.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b0_ns-c0e6a31c.pth


In [17]:
def criterion(outputs, labels):
    return nn.CrossEntropyLoss()(outputs, labels) # focal loss or BCEWithLogits 로 실험 필요 

In [18]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        images = data['image'].to(device)
        labels = data['label'].to(device)
        
        batch_size = images.size(0)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss = loss / CONFIG['n_accumulate']
            
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [19]:
def padded_cmap(solution, submission, padding_factor=5): # metric 
    new_rows = []
    for i in range(padding_factor):
        new_rows.append([1 for i in range(len(solution.columns))])
    new_rows = pd.DataFrame(new_rows)
    new_rows.columns = solution.columns
    padded_solution = pd.concat([solution, new_rows]).reset_index(drop=True).copy()
    padded_submission = pd.concat([submission, new_rows]).reset_index(drop=True).copy()
    score = average_precision_score(
        padded_solution.values,
        padded_submission.values,
        average='macro',
    )
    return score

In [20]:
@torch.inference_mode()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    LABELS = []
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        images = data['image'].to(device)
        labels = data['label_onehot'].to(device)
        
        batch_size = images.size(0)

        outputs = model(images)
        loss = criterion(outputs, labels)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        PREDS.append(F.softmax(outputs, dim=1).cpu().detach().numpy())
        LABELS.append(labels.cpu().detach().numpy())
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    LABELS = np.vstack(LABELS)
    PREDS = np.vstack(PREDS)
    val_cmap = padded_cmap(pd.DataFrame(LABELS), pd.DataFrame(PREDS))
    gc.collect()
    
    return epoch_loss, val_cmap

In [21]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    # To automatically log gradients
    wandb.watch(model, log_freq=100)
    
#     if torch.cuda.is_available():
#         print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
#     best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_cmap = 0
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss, val_epoch_cmap = valid_one_epoch(model, valid_loader, 
                                                       device=CONFIG['device'], 
                                                       epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Valid CMAP'].append(val_epoch_cmap)
        
        # Log the metrics
        wandb.log({"Train Loss": train_epoch_loss})
        wandb.log({"Valid Loss": val_epoch_loss})
        wandb.log({"Valid CMAP": val_epoch_cmap})
        
        # deep copy the model
        if val_epoch_cmap >= best_epoch_cmap:
            print(f"{b_}Validation CMAP Improved ({best_epoch_cmap} ---> {val_epoch_cmap})")
            best_epoch_cmap = val_epoch_cmap
            run.summary["Best CMAP Score"] = best_epoch_cmap
#             best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "CMAP{:.4f}_epoch{:.0f}.bin".format(best_epoch_cmap, epoch)
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best CMAP: {:.4f}".format(best_epoch_cmap))
    
    # load best model weights
#     model.load_state_dict(best_model_wts)
    
#     return model, history
    return history 

In [22]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [23]:
def prepare_loaders(df, fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = BirdCLEFDataset(df_train, target_sample_rate=CONFIG['sample_rate'], max_time=CONFIG['max_time'])
    valid_dataset = BirdCLEFDataset(df_valid, target_sample_rate=CONFIG['sample_rate'], max_time=CONFIG['max_time'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;">Prepare Dataloaders</span>

In [24]:
train_loader, valid_loader = prepare_loaders(df, fold=0)

<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;">Define Optimizer and Scheduler</span>

In [25]:
optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], 
                       weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;">Start Training</span>

In [26]:
run = wandb.init(project='BirdCLEF2023', 
                 config=CONFIG,
                 job_type='Train',
                 tags=[CONFIG['model_name']],
                 anonymous='must')

In [27]:
# model, history = run_training(model, optimizer, scheduler,
#                               device=CONFIG['device'],
#                               num_epochs=CONFIG['epochs'])
history = run_training(model, optimizer, scheduler,
                        device=CONFIG['device'],
                        num_epochs=CONFIG['epochs'])

100%|██████████| 27/27 [02:30<00:00,  5.59s/it, Epoch=1, LR=6.25e-5, Valid_Loss=3.91]


Validation CMAP Improved (0 ---> 0.5448120131866211)
Model Saved



100%|██████████| 27/27 [02:24<00:00,  5.35s/it, Epoch=2, LR=6.83e-6, Valid_Loss=3.49]


Validation CMAP Improved (0.5448120131866211 ---> 0.5790060756888165)
Model Saved



100%|██████████| 27/27 [02:23<00:00,  5.30s/it, Epoch=3, LR=1.73e-5, Valid_Loss=3.45]


...

100%|██████████| 27/27 [02:24<00:00,  5.34s/it, Epoch=8, LR=3.17e-5, Valid_Loss=2.79]


Validation CMAP Improved (0.6615969316005995 ---> 0.6634954705267658)
Model Saved



100%|██████████| 27/27 [02:22<00:00,  5.29s/it, Epoch=9, LR=9.04e-5, Valid_Loss=3.02]


100%|██████████| 27/27 [02:21<00:00,  5.24s/it, Epoch=10, LR=8.86e-5, Valid_Loss=3.08]



Training complete in 2h 13m 17s
Best CMAP: 0.6635


In [28]:
run.finish()

Train Loss,█▆▅▅▄▃▂▁▁▁
Valid CMAP,▁▃▃▅▇█████
Valid Loss,█▅▅▄▂▁▁▁▃▃
Best CMAP Score,0.6635
Train Loss,1.10623
Valid CMAP,0.65876
Valid Loss,3.07647


In [29]:
# This is just to display the W&B run page in this interactive session.
from IPython import display

# we create an IFrame and set the width and height
iF = display.IFrame(run.url, width=1080, height=720)
iF